<a href="https://colab.research.google.com/github/zaki-moh/MMA_fight_predictor/blob/main/mma_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from google.colab import files
uploaded = files.upload()

Saving large_dataset.csv to large_dataset (2).csv
Saving ufc-fighters-statistics.csv to ufc-fighters-statistics (3).csv


In [69]:
import pandas as pd
stats = pd.read_csv("ufc-fighters-statistics.csv")
historic_fights = pd.read_csv("large_dataset.csv")
# stats.head()
# stats.isnull().sum()
# print(stats.head(15))
# print(historic_fights.head(10))
#print(stats.columns)
stats.columns.tolist()

#

['name',
 'nickname',
 'wins',
 'losses',
 'draws',
 'height_cm',
 'weight_in_kg',
 'reach_in_cm',
 'stance',
 'date_of_birth',
 'significant_strikes_landed_per_minute',
 'significant_striking_accuracy',
 'significant_strikes_absorbed_per_minute',
 'significant_strike_defence',
 'average_takedowns_landed_per_15_minutes',
 'takedown_accuracy',
 'takedown_defense',
 'average_submissions_attempted_per_15_minutes']

In [51]:
# stats.info()
# stats.describe()
# stats.shape
# print(stats.columns.tolist())

In [70]:
historic_fights = historic_fights[['r_fighter', 'b_fighter', 'winner']].copy()
historic_fights['winner'] = historic_fights.apply(
    lambda row: row['r_fighter'] if row['winner'].lower() == 'red' else row['b_fighter'],
    axis = 1
)

stats['height_cm'] = stats['height_cm'].fillna(stats['height_cm'].median())
stats['date_of_birth'] = pd.to_datetime(stats['date_of_birth'], errors='coerce')
today = pd.to_datetime('today')
stats['age'] = (today - stats['date_of_birth']).dt.days // 365
stats['age'] = stats['age'].fillna(stats['age'].median())
mean_strikes = stats['significant_strikes_landed_per_minute'].mean()
stats['normalized_strikes'] = stats['significant_strikes_landed_per_minute'] / mean_strikes
stats['performance'] = (0.3 * stats['normalized_strikes']
                        + 0.3 * stats['significant_strike_defence'] / 100
                        + 0.2 * stats['takedown_accuracy'] / 100
                        + 0.2 * stats['takedown_defense'] / 100)
stats['strike_efficiency'] = (
    stats['significant_strikes_landed_per_minute']
    - stats['significant_strikes_absorbed_per_minute']
)
stats['grapple_efficiency'] = (
    0.5 * stats['takedown_accuracy'] / 100 + 0.5 * stats['takedown_defense'] / 100
)
stats['win_indicator'] = (stats['wins'] > stats['losses']).astype(int)
stats['win_ratio'] = stats['wins'] / (stats['wins'] + stats["losses"] + stats['draws'])
stats['win_ratio'] = stats['win_ratio'].fillna(0)

merged = historic_fights.merge(
    stats.add_prefix('r_'),
    left_on='r_fighter',
    right_on='r_name'
)
merged = merged.merge(
    stats.add_prefix('b_'),
    left_on='b_fighter',
    right_on='b_name'
)
print(merged.columns.tolist())

['r_fighter', 'b_fighter', 'winner', 'r_name', 'r_nickname', 'r_wins', 'r_losses', 'r_draws', 'r_height_cm', 'r_weight_in_kg', 'r_reach_in_cm', 'r_stance', 'r_date_of_birth', 'r_significant_strikes_landed_per_minute', 'r_significant_striking_accuracy', 'r_significant_strikes_absorbed_per_minute', 'r_significant_strike_defence', 'r_average_takedowns_landed_per_15_minutes', 'r_takedown_accuracy', 'r_takedown_defense', 'r_average_submissions_attempted_per_15_minutes', 'r_age', 'r_normalized_strikes', 'r_performance', 'r_strike_efficiency', 'r_grapple_efficiency', 'r_win_indicator', 'r_win_ratio', 'b_name', 'b_nickname', 'b_wins', 'b_losses', 'b_draws', 'b_height_cm', 'b_weight_in_kg', 'b_reach_in_cm', 'b_stance', 'b_date_of_birth', 'b_significant_strikes_landed_per_minute', 'b_significant_striking_accuracy', 'b_significant_strikes_absorbed_per_minute', 'b_significant_strike_defence', 'b_average_takedowns_landed_per_15_minutes', 'b_takedown_accuracy', 'b_takedown_defense', 'b_average_submi

In [40]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_cols = ['height_cm', 'age', 'performance', 'strike_efficiency', 'grapple_efficiency']

stats[numeric_cols] = scaler.fit_transform(stats[numeric_cols])

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X = stats[['strike_efficiency', 'grapple_efficiency', 'height_cm', 'age', 'performance']]
y = stats['win_indicator']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8424124513618677
              precision    recall  f1-score   support

           0       0.49      0.56      0.52       160
           1       0.92      0.90      0.91       868

    accuracy                           0.84      1028
   macro avg       0.71      0.73      0.71      1028
weighted avg       0.85      0.84      0.85      1028



In [42]:
def predictWinner(fighter_A_Name, fighter_B_Name):
  fighter_A = stats.loc[stats['name'] == fighter_A_Name].squeeze()
  fighter_B = stats.loc[stats['name'] == fighter_B_Name].squeeze()

  if fighter_A.empty or fighter_B.empty:
    print("One or both fighters not found in the dataset.")
    return

  features = ['strike_efficiency', 'grapple_efficiency', 'height_cm', 'age', 'performance']

  feature_A = fighter_A[features].values.reshape(1, -1)
  feature_B = fighter_B[features].values.reshape(1, -1)

  A_prob = model.predict_proba(feature_A)[0][1]
  B_prob = model.predict_proba(feature_B)[0][1]

  print(f"{fighter_A_Name} vs {fighter_B_Name}")
  print(f"{fighter_A_Name} likelyhood to win {A_prob:.2%}")
  print(f"{fighter_B_Name} likelyhood to win {A_prob:.2%}")

  if A_prob > B_prob:
    print(f"Predicted winner {fighter_A_Name}")
  else:
    print(f"Predicted winner {fighter_B_Name}")




In [43]:
#Test
predictWinner("Khabib Nurmagomedov", "Conor McGregor")

Khabib Nurmagomedov vs Conor McGregor
Khabib Nurmagomedov likelyhood to win 100.00%
Conor McGregor likelyhood to win 100.00%
Predicted winner Conor McGregor


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
